In [1]:
using Plots, Optim, LinearAlgebra, LaTeXStrings

In [51]:
abstract type BoseHubbardModel end

struct MeanField <: BoseHubbardModel
    n_max :: Int64
    a :: Matrix{Float64}
    n :: Matrix{Float64}

    MeanField(n_max) = new(n_max, diagm(1 => sqrt.(1:n_max)), diagm(0 => 0:n_max))
end

function get_hamiltonian(model :: MeanField, t, mu, V, z, psi)
    H = -mu * model.n + 0.5 * model.n * (model.n - I) + 0.5 * V * (2 * z * psi[2] * model.n - z * psi[2] ^ 2 * I) - t * z * psi[1] * (model.a + model.a') + t * z * psi[1]^2 * I
    return H 
end

function get_order_parameter(model, t, mu, V, z)
    E_gs(psi) = eigvals(get_hamiltonian(model, t, mu, V, z, psi))[1]
    return Optim.minimizer(optimize(E_gs, zeros(2)))
end

# find N for the ground state
function get_num_particles(model, t, mu, V, z)
    psi, phi = get_order_parameter(model, t, mu, V, z)
    gs = eigvecs(get_hamiltonian(model, t, mu, V, z, psi))[:, 1]
    
    return gs' * model.n * gs
end

get_num_particles (generic function with 1 method)

In [57]:
model = MeanField(10)
z, size = 2, 100
t, mu, V = 0.1, 2, 1

for (psi, phi) in zip(-100:0.1:100, -100:0.1:100)
    H = get_hamiltonian(model, t, mu, V, 2, [-1, 0])
    if(isnothing(H) || any(isnan, H) || any(isinf, H))
        print(H)
        break;
    end
end
# # get_order_parameter(model, t, mu, V, 1)

In [49]:
model = MeanField(10)

z, size = 2, 100
t = range(start = 0, stop = 0.1, length = size)
mu = range(start = 0, stop = 3, length = size)
V = 1

order_param = zeros((size, size, 2))
num_particles = zeros((size, size))

for k1 in 1:size, k2 in 1:size
    order_param[k2, k1, :] .= abs.(get_order_parameter(model, t[k1], mu[k2], V, z))
    num_particles[k2, k1] = abs.(get_num_particles(model, t[k1], mu[k2], V, z))
end


ArgumentError: ArgumentError: matrix contains Infs or NaNs